In [1]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from data_processing import DatasetProcessor
from bert_data_processing import BERTDataProcessor
from bert_evaluator import BERTEvaluator
from bert_trainer import BERTTrainer
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG

INFO:datasets:PyTorch version 2.5.1+cu124 available.
INFO:datasets:Polars version 1.9.0 available.
INFO:datasets:Duckdb version 1.1.3 available.
INFO:datasets:TensorFlow version 2.18.0 available.
INFO:datasets:JAX version 0.4.33 available.


INFO:config.py:54: PyTorch version 2.5.1+cu124 available.
INFO:config.py:66: Polars version 1.9.0 available.
INFO:config.py:77: Duckdb version 1.1.3 available.
INFO:config.py:112: TensorFlow version 2.18.0 available.
INFO:config.py:125: JAX version 0.4.33 available.


In [3]:
def train_model(model_name, dataset_name, epochs=3, batch_size=16):
    if dataset_name.lower() == 'combined':
      ewt_processor = DatasetProcessor('ewt', from_json=True)
      edt_processor = DatasetProcessor('edt', from_json=True)
      ewt_dataset = ewt_processor.dataset
      edt_dataset = edt_processor.dataset
      dataset = DatasetProcessor.combine_datasetdicts(ewt_dataset, edt_dataset)
    elif dataset_name.lower() in ['ewt', 'edt']:
      processor = DatasetProcessor(dataset_name, from_json=True)
      dataset = processor.dataset

    print(f'{dataset_name.upper()} andmestik laetud')
    bert_processor = BERTDataProcessor(model_name)
    evaluator = BERTEvaluator(all_tags=ALL_TAGS)

    tokenized_dataset = bert_processor.tokenize_dataset(dataset)

    trainer = BERTTrainer(model_name=model_name, idx2tag=IDX2TAG, tag2idx=TAG2IDX, evaluator=evaluator)

    model, model_trainer = trainer.finetune_model(processor=bert_processor, tokenized_dataset=tokenized_dataset, epochs=epochs, batch_size=batch_size, early_stop_patience=3, output_dir=f'./results/{model_name.split("/")[1]}/{dataset_name}')

    results = evaluator.evaluate_and_print(tokenized_dataset['test'], model_trainer)

    if dataset_name.lower() == 'combined':
      tokenized_ewt = bert_processor.tokenize_dataset(ewt_dataset)
      tokenized_edt = bert_processor.tokenize_dataset(edt_dataset)

      print(f'Kombineeritud andmestikul treenitud {model_name} EWT testandmestikul')
      ewt_results = evaluator.evaluate_and_print(tokenized_ewt['test'], model_trainer)
      print(f'Kombineeritud andmestikul treenitud {model_name} EDT testandmestikul')
      edt_results = evaluator.evaluate_and_print(tokenized_edt['test'], model_trainer)

      return model, model_trainer, (results, ewt_results, edt_results)

    return model, model_trainer, results

def train_all():
  dataset_configs = {
        'ewt': {'epochs': 10},
        'edt': {'epochs': 5},
        'combined': {'epochs': 3}
    }

  model_names = ["tartuNLP/EstRoBERTa", "tartuNLP/EstBERT"]
  results_dict = {}

  for model_name in model_names:
    model_results = {}
    for dataset_name, config in dataset_configs.items():
      model, model_trainer, results = train_model(model_name, dataset_name, epochs=config['epochs'])
      model_results[dataset_name] = results
    results_dict[model_name] = model_results

  return results_dict

In [ ]:
def main():
    results = train_all()
    try:
      for model_name, model_results in results.items():
        for dataset_name, dataset_results in model_results.items():
          print(f"{model_name}, {dataset_name}")
          print(dataset_results)
          print()
    except:
      pass

In [ ]:
main()

EWT dataset loaded


tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.092700,0.133657,0.729124,0.709000,0.718921,0.966568,0.500000,0.397059,0.442623,68,0.589916,0.780000,0.671770,450,0.795455,0.542636,0.645161,645,0.000000,0.000000,0.000000,12,0.478941,0.768340,0.590067,518,0.937037,0.871662,0.903168,2322,0.505115,0.432640,0.466077,913,0.000000,0.000000,0.000000,72
2,0.039900,0.147840,0.731600,0.731600,0.731600,0.968088,0.382979,0.529412,0.444444,68,0.601562,0.684444,0.640333,450,0.705785,0.662016,0.683200,645,0.000000,0.000000,0.000000,12,0.542113,0.683398,0.604611,518,0.935181,0.900947,0.917745,2322,0.490545,0.483023,0.486755,913,0.000000,0.000000,0.000000,72
3,0.020000,0.171511,0.740862,0.725600,0.733151,0.968768,0.333333,0.367647,0.349650,68,0.580000,0.773333,0.662857,450,0.745597,0.590698,0.659170,645,0.000000,0.000000,0.000000,12,0.542222,0.706564,0.613579,518,0.936572,0.896641,0.916172,2322,0.524631,0.466594,0.493913,913,0.000000,0.000000,0.000000,72


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2025-02-12 13:58:10] tartuNLP/EstBERT treenimine lõpetatud
Kokku kulus: 888.35 sekundit (0.25 tundi)
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.48091603053435117, 'recall': 0.44680851063829785, 'f1': 0.4632352941176471, 'number': 141}
GEP {'precision': 0.6, 'recall': 0.765375854214123, 'f1': 0.6726726726726726, 'number': 439}
LOC {'precision': 0.7223340040241448, 'recall': 0.7237903225806451, 'f1': 0.7230614300100703, 'number': 496}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7300546448087432, 'recall': 0.6958333333333333, 'f1': 0.7125333333333334, 'number': 960}
PER {'precision': 0.9399938137952366, 'recall': 0.8880771478667446, 'f1': 0.9132982719759579, 'number': 3422}
PROD {'precision': 0.4866023579849946, 'recall': 0.47689075630252103, 'f1': 0.4816976127320955, 'number': 952}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7846546498644121
overall_recall 0.7664381427235899
overall_f1 0.7754394261842831
overall_accuracy 0.9703505703234707

Nervaluate tulemused
Strict {'correct': 4919, 'incorrect': 967, 'partial': 0,

In [4]:
estroberta_edt, estroberta_edt_trainer, estroberta_edt_results = train_model('tartuNLP/EstRoBERTa', 'edt', epochs=6)

EDT andmestik laetud


tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.073100,0.082257,0.732492,0.724248,0.728346,0.977147,0.588235,0.449438,0.509554,89,0.615265,0.769981,0.683983,513,0.829187,0.602410,0.697837,830,0.000000,0.000000,0.000000,9,0.626050,0.758270,0.685846,393,0.852459,0.925267,0.887372,1686,0.395735,0.316888,0.351949,527,0.000000,0.000000,0.000000,40
2,0.047300,0.076675,0.746067,0.742598,0.744329,0.979228,0.539474,0.460674,0.496970,89,0.586755,0.863548,0.698738,513,0.838772,0.526506,0.646928,830,0.000000,0.000000,0.000000,9,0.694581,0.717557,0.705882,393,0.886996,0.926453,0.906295,1686,0.491803,0.512334,0.501859,527,0.000000,0.000000,0.000000,40
3,0.028500,0.090878,0.737754,0.729631,0.733670,0.978795,0.771930,0.494382,0.602740,89,0.573840,0.795322,0.666667,513,0.758089,0.592771,0.665314,830,0.000000,0.000000,0.000000,9,0.603896,0.709924,0.652632,393,0.885532,0.903915,0.894629,1686,0.531674,0.445920,0.485036,527,0.000000,0.000000,0.000000,40
4,0.017600,0.099343,0.744760,0.730365,0.737492,0.978963,0.600000,0.505618,0.548780,89,0.561589,0.826511,0.668770,513,0.790698,0.614458,0.691525,830,0.000000,0.000000,0.000000,9,0.617257,0.709924,0.660355,393,0.897862,0.896797,0.897329,1686,0.544304,0.407970,0.466377,527,0.000000,0.000000,0.000000,40
5,0.008800,0.117439,0.738020,0.742354,0.740181,0.978566,0.666667,0.561798,0.609756,89,0.544192,0.840156,0.660536,513,0.822901,0.649398,0.725926,830,0.000000,0.000000,0.000000,9,0.572000,0.727735,0.640538,393,0.916974,0.884342,0.900362,1686,0.523179,0.449715,0.483673,527,0.000000,0.000000,0.000000,40


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

[2025-02-26 10:31:37] tartuNLP/EstRoBERTa treenimine lõpetatud
Kokku kulus: 1807.05 sekundit (0.50 tundi)
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.6964285714285714, 'recall': 0.4105263157894737, 'f1': 0.5165562913907285, 'number': 95}
GEP {'precision': 0.6513605442176871, 'recall': 0.8148936170212766, 'f1': 0.7240075614366729, 'number': 470}
LOC {'precision': 0.8348623853211009, 'recall': 0.5824, 'f1': 0.6861451460885958, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.736998514115899, 'recall': 0.6785225718194254, 'f1': 0.7065527065527065, 'number': 731}
PER {'precision': 0.9134615384615384, 'recall': 0.9549825174825175, 'f1': 0.9337606837606838, 'number': 2288}
PROD {'precision': 0.5621890547263682, 'recall': 0.5355450236966824, 'f1': 0.5485436893203883, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.8016006739679865
overall_recall 0.7844187963726298
overall_f1 0.7929166666666666
overall_accuracy 0.9817464570340558

Nervaluate tulemused
Strict {'correct': 3806, 'incorrect': 818, 'partial': 0, 

In [5]:
estroberta_combined, estroberta_combined_trainer, estroberta_combined_results = train_model('tartuNLP/EstRoBERTa', 'combined', epochs=6)

COMBINED andmestik laetud


Map:   0%|          | 0/30045 [00:00<?, ? examples/s]

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstRoBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Kombineeritud andmestikul treenitud tartuNLP/EstRoBERTa EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.8085106382978723, 'recall': 0.6909090909090909, 'f1': 0.7450980392156863, 'number': 55}
GEP {'precision': 0.5625, 'recall': 0.9, 'f1': 0.6923076923076923, 'number': 60}
LOC {'precision': 0.4368932038834951, 'recall': 0.7258064516129032, 'f1': 0.5454545454545454, 'number': 62}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.5379746835443038, 'recall': 0.4913294797687861, 'f1': 0.5135951661631419, 'number': 173}
PER {'precision': 0.9409090909090909, 'recall': 0.8173741362290227, 'f1': 0.8748019017432646, 'number': 1013}
PROD {'precision': 0.4385245901639344, 'recall': 0.34516129032258064, 'f1': 0.3862815884476535, 'number': 310}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.743573264781491
overall_recall 0.6915720263000598
overall_f1 0.7166305357695881
overall_accuracy 0.9615103976650857

Nervaluate tulemused
Strict {'correct': 1157, 'incorrect': 260, 'partial': 0, 'missed': 256, '

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.620253164556962, 'recall': 0.5157894736842106, 'f1': 0.5632183908045977, 'number': 95}
GEP {'precision': 0.6721014492753623, 'recall': 0.7893617021276595, 'f1': 0.726027397260274, 'number': 470}
LOC {'precision': 0.7457337883959044, 'recall': 0.6992, 'f1': 0.7217175887696119, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7506738544474394, 'recall': 0.761969904240766, 'f1': 0.7562797012898846, 'number': 731}
PER {'precision': 0.9401518535060295, 'recall': 0.9200174825174825, 'f1': 0.9299757013474707, 'number': 2288}
PROD {'precision': 0.6164383561643836, 'recall': 0.5687203791469194, 'f1': 0.5916187345932622, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.809136420525657
overall_recall 0.7994641384995878
overall_f1 0.8042712004976155
overall_accuracy 0.9819516811273643

Nervaluate tulemused
Strict {'correct': 3879, 'incorrect': 761, 'partial': 0, 'mi

In [6]:
estbert_combined, estbert_combined_trainer, estbert_combined_results = train_model('tartuNLP/EstBERT', 'combined', epochs=6)

COMBINED andmestik laetud


Map:   0%|          | 0/30045 [00:00<?, ? examples/s]

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.5, 'recall': 0.49122807017543857, 'f1': 0.49557522123893805, 'number': 57}
GEP {'precision': 0.3508771929824561, 'recall': 0.7272727272727273, 'f1': 0.47337278106508873, 'number': 55}
LOC {'precision': 0.34375, 'recall': 0.6, 'f1': 0.4370860927152318, 'number': 55}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.6862745098039216, 'recall': 0.45454545454545453, 'f1': 0.546875, 'number': 231}
PER {'precision': 0.9619047619047619, 'recall': 0.8301369863013699, 'f1': 0.8911764705882352, 'number': 1095}
PROD {'precision': 0.4142857142857143, 'recall': 0.4005524861878453, 'f1': 0.40730337078651685, 'number': 362}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.7291666666666666
overall_recall 0.6792452830188679
overall_f1 0.7033212391850404
overall_accuracy 0.9478434732933447

Nervaluate tulemused
Strict {'correct': 1260, 'incorrect': 301, 'partial': 0, 'missed': 294, 'spurious': 177, 'po

Seqeval tulemused
EVE {'precision': 0.4583333333333333, 'recall': 0.39285714285714285, 'f1': 0.4230769230769231, 'number': 84}
GEP {'precision': 0.674074074074074, 'recall': 0.7109375, 'f1': 0.6920152091254753, 'number': 384}
LOC {'precision': 0.7473002159827213, 'recall': 0.7845804988662132, 'f1': 0.7654867256637169, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7185385656292287, 'recall': 0.7283950617283951, 'f1': 0.7234332425068121, 'number': 729}
PER {'precision': 0.9377797672336616, 'recall': 0.9003008165019338, 'f1': 0.9186581889936417, 'number': 2327}
PROD {'precision': 0.4826086956521739, 'recall': 0.5644067796610169, 'f1': 0.5203125, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7827877736830696
overall_recall 0.7913653298268682
overall_f1 0.7870531822144726
overall_accuracy 0.9749203008935723

Nervaluate tulemused
Strict {'correct': 3611, 'incorrect': 724, 'partial': 0, 'miss